In [9]:
import numpy as np
import pandas as pd
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import os
import sys
from scipy.signal import butter, filtfilt
import csv

In [10]:
points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ,29, 30, 31, 32]

In [11]:
### Markerless 처리 ###

def calibrating(rootPath):
    # 캘리브레이션 설정
    chessboard_size = (5, 4)  # 체커보드 패턴의 내부 코너 수 (6x5)
    square_size = 3.5  # 체커보드에서 사각형 한 칸의 크기 (단위는 미터나 cm)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # 3D 점들과 2D 점들 저장을 위한 배열
    objpoints = []  # 3D 공간에서의 체커보드 패턴 점
    imgpoints = []  # 이미지 평면에서의 체커보드 패턴 점

    # 체커보드의 3D 점을 정의
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    objp *= square_size  # 각 점에 사각형 크기를 곱함

    # 캘리브레이션 이미지 디렉토리
    calibration_dir = f"{rootPath}\\calibration_img"

    # 캘리브레이션 이미지 가져오기
    calibration_images = [f for f in os.listdir(calibration_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    # 체커보드 코너 찾기 및 캘리브레이션 수행
    for fname in calibration_images:
        img = cv2.imread(os.path.join(calibration_dir, fname))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 체커보드 코너 찾기
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        # 코너가 발견되면
        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

    # 카메라 캘리브레이션 수행
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return ret, camera_matrix, dist_coeffs, rvecs, tvecs

def get_xyz(detection_result, rootPath):
    res = []
    
    # 캘리브레이션된 카메라 매트릭스와 왜곡 계수
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = calibrating(rootPath)

    for i in points:
        try:
            # 포즈 추정 결과에서 x, y, z 좌표 추출 (이미지 좌표계 기준)
            x = detection_result.pose_landmarks[0][i].x
            y = detection_result.pose_landmarks[0][i].y
            z = detection_result.pose_landmarks[0][i].z
            
            # 왜곡 보정 (이미지 좌표계에서 보정된 좌표로 변환)
            undistorted_points = cv2.undistortPoints(
                np.array([[x, y]], dtype=np.float32), 
                camera_matrix, dist_coeffs
            )
            
            # 보정된 좌표에서 x, y 값을 업데이트
            undistorted_x, undistorted_y = undistorted_points[0][0]
            
            # 실제 월드 좌표를 계산할 수 있는 방법은 필요에 따라 추가 가능 (z 축 포함)
            # 여기서는 단순히 보정된 이미지 좌표만 사용
            res.append(undistorted_x)
            res.append(undistorted_y)
            res.append(z)  # z 값은 보정하지 않고 그대로 사용

        except Exception as e:
            # 좌표가 존재하지 않을 경우 기본값으로 -10을 추가
            res.extend([-10, -10, -10])

    return res


def draw_landmarks_on_image(rgb_image, detection_result): # 랜드마크 보여주는 함수, 굳이?
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])

        solutions.drawing_utils.draw_landmarks(
        annotated_image,
        pose_landmarks_proto,
        solutions.pose.POSE_CONNECTIONS,
        solutions.drawing_styles.get_default_pose_landmarks_style()
        )
    return annotated_image


In [12]:
rootPath = os.getcwd()
# 일괄처리 코드
# Create base options for the PoseLandmarker

base_options = python.BaseOptions(model_asset_path=f"{rootPath}\\pose_landmarker_heavy.task")
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True
)
detector = vision.PoseLandmarker.create_from_options(options)

# Define input and output directories
input_dir = f"{rootPath}\\markerless_video"

# Get all video files in the input directory
video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mov', '.mp4', '.avi'))]

# Process each video file
for video_file in video_files:
    video_path = os.path.join(input_dir, video_file)
    
    # Open the video file using OpenCV
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2  # Reduce frame size by half
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2

    # Initialize an empty list to store DataFrame rows
    rows = []

    while cap.isOpened():
        ret, frame = cap.read()

        # End loop if the video ends or reading fails
        if not ret:
            break

        frame = cv2.resize(frame, (frame_width, frame_height))

        # Convert OpenCV frame to Mediapipe image
        mp_frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detect pose landmarks
        detection_result = detector.detect(mp_frame)

        # Get xyz coordinates
        d = get_xyz(detection_result, rootPath)

        # Add coordinates to rows list
        rows.append(d)

        # Optional: Visualize landmarks
        annotated_frame = draw_landmarks_on_image(frame, detection_result)
        cv2.imshow('Pose Detection', annotated_frame)   # landmarks 적용 video 보여주는 코드

        # Check for 'q' key press to exit loop and stop processing
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting process...")
            cap.release()
            cv2.destroyAllWindows()
            sys.exit()  # 전체 스크립트 종료

    # Release the video and close windows
    cap.release()
    cv2.destroyAllWindows()